In [272]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_circular_economy as mc
from model_circular_economy import CircularEconomy
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)




<module 'model_circular_economy' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_circular_economy.py'>

In [275]:
df_tmp = sa.model_attributes.transfer_df_variables(
    df_complete,
    df_project_afolu,
    model_circecon.integration_variables
)

In [277]:
os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv")

'/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/fake_data/fake_data_complete.csv'

In [266]:
df_complete = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_waste = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_circular_economy.csv"))
df_socio = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_socioeconomic.csv"))
model_afolu = ma.AFOLU(sa.model_attributes);
model_circecon = mc.CircularEconomy(sa.model_attributes);
df_project_afolu = model_afolu.project(df_complete)
# pass integrated df
df_waste_integrated = pd.merge(df_complete, df_project_afolu)
df_project_ce = model_circecon.project(df_waste_integrated);
df_waste_integrated = sf.merge_output_df_list([df_waste_integrated, df_project_ce], sa.model_attributes, "concatenate")

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:341: RuntimeWarning: divide by zero encountered in true_divide
  arr_agrc_yield = np.array([(vec_initial_area[ind_crop]*vec_agrc_frac_cropland_area*arr_agrc_yield_factors[0]) for k in range(n_tp)])
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:378: RuntimeWarning: invalid value encountered in multiply
  # calculate required increase in transition probabilities
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:549: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [279]:
x = 2
x = 6 if 4 > 5 else x
x

2

In [263]:
?argparse.ArgumentParser.add_argument

Signature: argparse.ArgumentParser.add_argument(self, *args, **kwargs)
Docstring:
add_argument(dest, ..., name=value, ...)
add_argument(option_string, option_string, ..., name=value, ...)
File:      ~/anaconda3/lib/python3.8/argparse.py
Type:      function


In [ ]:
import support_functions as sf
import data_structures as ds
from model_socioeconomic import Socioeconomic
import pandas as pd
import numpy as np
import time


#########################
###                   ###
###     IPPU MODEL    ###
###                   ###
#########################

class IPPU:

    def __init__(self, attributes: ds.ModelAttributes):

        self.model_attributes = attributes
        self.required_dimensions = self.get_required_dimensions()
        self.required_subsectors, self.required_base_subsectors = self.get_required_subsectors()
        self.required_variables, self.output_variables = self.get_ippu_input_output_fields()

        ##  set some model fields to connect to the attribute tables

        # ippu model variables
        self.modvar_ippu_change_net_imports = "Change to Net Imports of Recyclable Products"
        self.modvar_ippu_ef_ch4_per_prod = ":math:\\text{CH}_4 Production Emission Factor"
        self.modvar_ippu_ef_co2_per_gdp = ":math:\\text{CO}_2 GDP Emission Factor"
        self.modvar_ippu_ef_co2_per_prod = ":math:\\text{CO}_2 Production Emission Factor"
        self.modvar_ippu_ef_hfc_per_gdp = "HFC Emission GDP Factor Placeholder"
        self.modvar_ippu_ef_hfc_per_prod = "HFC Emission Production Factor Placeholder"
        self.modvar_ippu_ef_n2o_per_gdp = ":math:\\text{N}_2\\text{O} GDP Emission Factor"
        self.modvar_ippu_ef_n2o_per_prod = ":math:\\text{N}_2\\text{O} Production Emission Factor"
        self.modvar_ippu_ef_nf3_per_prod = ":math:\\text{NF}_3 Production Emission Factor"
        self.modvar_ippu_ef_pfc_per_gdp = "PFC Emission GDP Factor Placeholder"
        self.modvar_ippu_ef_pfc_per_prod = "PFC Emission Production Factor Placeholder"
        self.modvar_ippu_ef_sf6_per_gdp = ":math:\\text{SF}_6 GDP Emission Factor"
        self.modvar_ippu_ef_sf6_per_prod = ":math:\\text{SF}_6 Production Emission Factor"
        self.modvar_ippu_elast_ind_prod_to_gdp = "Elasticity of Industrial Production to GDP"
        self.modvar_ippu_emissions_ch4 = ":math:\\text{CH}_4 Emissions from Industry"
        self.modvar_ippu_emissions_co2 = ":math:\\text{CO}_2 Emissions from Industry"
        self.modvar_ippu_emissions_hfc = "HFC Emissions Placeholder"
        self.modvar_ippu_emissions_n2o = ":math:\\text{N}_2\\text{O} Emissions from Industry"
        self.modvar_ippu_emissions_pfc = "PFC Emissions Placeholder"
        self.modvar_ippu_emissions_nf3 = ":math:\\text{NF}_3 Emissions from Industry"
        self.modvar_ippu_emissions_sf6 = ":math:\\text{SF}_6 Emissions from Industry"
        self.modvar_ippu_prod_qty_init = "Initial Industrial Production"
        self.modvar_ippu_wwf_cod = "COD Wastewater Factor"
        self.modvar_ippu_wwf_vol = "Wastewater Production Factor"
        self.modvar_ippu_qty_total_production = "Industrial Production"

        # variables from other sectors
        self.modvar_waso_waste_total_recycled = "Total Waste Recycled"

        # add other model classes
        self.model_socioeconomic = Socioeconomic(self.model_attributes)

        # optional integration variables (uses calls to other model classes)
        self.integration_variables = self.set_integrated_variables()

        ##  MISCELLANEOUS VARIABLES
        self.time_periods, self.n_time_periods = self.model_attributes.get_time_periods()



    ##  FUNCTIONS FOR MODEL ATTRIBUTE DIMENSIONS

    def check_df_fields(self, df_ippu_trajectories):
        check_fields = self.required_variables
        # check for required variables
        if not set(check_fields).issubset(df_ippu_trajectories.columns):
            set_missing = list(set(check_fields) - set(df_ippu_trajectories.columns))
            set_missing = sf.format_print_list(set_missing)
            raise KeyError(f"IPPU projection cannot proceed: The fields {set_missing} are missing.")

    def get_required_subsectors(self):
        subsectors = list(sf.subset_df(self.model_attributes.dict_attributes["abbreviation_subsector"].table, {"sector": ["IPPU"]})["subsector"])
        subsectors_base = subsectors.copy()
        subsectors += ["Economy", "General"]
        return subsectors, subsectors_base

    def get_required_dimensions(self):
        ## TEMPORARY - derive from attributes later
        required_doa = [self.model_attributes.dim_time_period]
        return required_doa

    def get_ippu_input_output_fields(self):
        required_doa = [self.model_attributes.dim_time_period]
        required_vars, output_vars = self.model_attributes.get_input_output_fields(self.required_subsectors)
        return required_vars + self.get_required_dimensions(), output_vars

    def set_integrated_variables(self):
        list_vars_required_for_integration = [
            self.modvar_waso_waste_total_recycled
        ]

        return list_vars_required_for_integration


    ######################################
    #    SUBSECTOR SPECIFIC FUNCTIONS    #
    ######################################


    # project industrial production—broken out so that other sectors can call it
    def project_industrial_production(self,
        df_ippu_trajectories: pd.DataFrame,
        vec_rates_gdp: np.ndarray,
        dict_dims: dict = None,
        n_projection_time_periods: int = None,
        projection_time_periods: list = None
    ) -> np.ndarray:
        """
            project_industrial_production can be called from other sectors to simplify calculation of industrial production. If any of dict_dims, n_projection_time_periods, or projection_time_periods are unspecified (expected if ran outside of IPPU.project()), self.model_attributes.check_projection_input_df wil be run

            dict_dims: dict of dimensions (returned from check_projection_input_df). Default is None.

            n_projection_time_periods: integer giving number of time periods (returned from check_projection_input_df). Default is None.

            projection_time_periods: list of time periods (returned from check_projection_input_df). Default is None.
        """
        # allows production to be run outside of the project method
        if type(None) in set([type(x) for x in [dict_dims, n_projection_time_periods, projection_time_periods]]):
            dict_dims, df_ippu_trajectories, n_projection_time_periods, projection_time_periods = self.model_attributes.check_projection_input_df(df_ippu_trajectories, True, True, True)

        # get initial production and apply elasticities to gdp to calculate growth in production
        array_ippu_prod_init_by_cat = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.modvar_ippu_prod_qty_init, False, return_type = "array_base", var_bounds = (0, np.inf), expand_to_all_cats = True)
        array_ippu_elasticity_prod_to_gdp = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.modvar_ippu_elast_ind_prod_to_gdp, False, return_type = "array_base", expand_to_all_cats = True)
        array_ippu_ind_growth = sf.project_growth_scalar_from_elasticity(vec_rates_gdp, array_ippu_elasticity_prod_to_gdp, False, "standard")
        array_ippu_ind_prod = array_ippu_prod_init_by_cat[0]*array_ippu_ind_growth

        return array_ippu_ind_prod


    # project method for IPPU
    def project(self, df_ippu_trajectories: pd.DataFrame) -> pd.DataFrame:

        # make sure socioeconomic variables are added and
        df_ippu_trajectories, df_se_internal_shared_variables = self.model_socioeconomic.project(df_ippu_trajectories)
        # check that all required fields are contained—assume that it is ordered by time period
        self.check_df_fields(df_ippu_trajectories)
        dict_dims, df_ippu_trajectories, n_projection_time_periods, projection_time_periods = self.model_attributes.check_projection_input_df(df_ippu_trajectories, True, True, True)

        ##  CATEGORY AND ATTRIBUTE INITIALIZATION
        pycat_gnrl = self.model_attributes.get_subsector_attribute("General", "pycategory_primary")
        pycat_ippu = self.model_attributes.get_subsector_attribute("IPPU", "pycategory_primary")
        pycat_waso = self.model_attributes.get_subsector_attribute("Solid Waste", "pycategory_primary")
        # attribute tables
        attr_gnrl = self.model_attributes.dict_attributes[pycat_gnrl]
        attr_ippu = self.model_attributes.dict_attributes[pycat_ippu]
        attr_waso = self.model_attributes.dict_attributes[pycat_waso]


        ##  ECON/GNRL VECTOR AND ARRAY INITIALIZATION

        # get some vectors
        vec_gdp = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_econ_gdp, False, return_type = "array_base")
        vec_pop = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_gnrl_pop_total, False, return_type = "array_base")
        array_pop = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_gnrl_subpop, False, return_type = "array_base")
        vec_gdp_per_capita = np.array(df_se_internal_shared_variables["vec_gdp_per_capita"])
        vec_rates_gdp = np.array(df_se_internal_shared_variables["vec_rates_gdp"].dropna())
        vec_rates_gdp_per_capita = np.array(df_se_internal_shared_variables["vec_rates_gdp_per_capita"].dropna())


        ##  OUTPUT INITIALIZATION
        df_out = [df_ippu_trajectories[self.required_dimensions].copy()]


        ######################################################
        #    INDUSTRIAL PRODUCTION + RECYCLING ADJUSTMENT    #
        ######################################################

        # initialize production + initialize change to net imports as 0 (reduce categories later)
        array_ippu_production = self.project_industrial_production(df_ippu_trajectories, vec_rates_gdp, dict_dims, n_projection_time_periods, projection_time_periods)
        array_ippu_change_net_imports = np.zeros((array_ippu_production.shape[0], attr_ippu.n_key_values))


        ##  PERFORM THE RECYCLING ADJUSTMENT (if recycling data are provided from the waste model)

        array_ippu_recycled = self.model_attributes.get_optional_or_integrated_standard_variable(df_ippu_trajectories, self.modvar_waso_waste_total_recycled, None, True, "array_base")
        if type(array_ippu_recycled) != type(None):
            # if recycling totals are passed from the waste model, convert to ippu categories
            cats_waso_recycle = sa.model_attributes.get_variable_categories(self.modvar_waso_waste_total_recycled)
            dict_repl = attr_waso.field_maps[f"{pycat_waso}_to_{pycat_ippu}"]
            cats_ippu_recycle = [ds.clean_schema(dict_repl[x]) for x in cats_waso_recycle]
            array_ippu_recycled_waste = self.model_attributes.merge_array_var_partial_cat_to_array_all_cats(
                array_ippu_recycled[1],
                None,
                output_cats = cats_ippu_recycle,
                output_subsec = "IPPU"
            )
            # units correction to ensure consistency from waso -> ippu
            factor_ippu_waso_recycle_to_ippu_recycle = self.model_attributes.get_mass_equivalent(
                self.model_attributes.get_variable_characteristic(self.modvar_waso_waste_total_recycled, "$UNIT-MASS$"),
                self.model_attributes.get_variable_characteristic(self.modvar_ippu_prod_qty_init, "$UNIT-MASS$")
            )
            array_ippu_recycled_waste *= factor_ippu_waso_recycle_to_ippu_recycle
            array_ippu_production += array_ippu_recycled_waste
            # next, check for industrial categories whose production is affected by recycling, then adjust downwards
            vec_ippu_cats_to_adjust_from_recycling = [ds.clean_schema(x) for x in sa.model_attributes.get_ordered_category_attribute("IPPU", "target_recycling_cat_industry_to_adjust")]
            w = [i for i in range(len(vec_ippu_cats_to_adjust_from_recycling)) if (vec_ippu_cats_to_adjust_from_recycling[i] != "none") and (vec_ippu_cats_to_adjust_from_recycling[i] in attr_ippu.key_values)]
            if len(w) > 0:
                array_ippu_recycled_waste_adj = array_ippu_recycled_waste[:, w].copy()
                array_ippu_recycled_waste_adj = self.model_attributes.merge_array_var_partial_cat_to_array_all_cats(
                    array_ippu_recycled_waste_adj,
                    None,
                    output_cats = np.array(vec_ippu_cats_to_adjust_from_recycling)[w],
                    output_subsec = "IPPU"
                )
                # inititialize production, then get change to net imports (anything negative) and reduce virgin production accordingly
                array_ippu_production = array_ippu_production - array_ippu_recycled_waste_adj
                array_ippu_change_net_imports = sf.vec_bounds(array_ippu_production, (-np.inf, 0))
                array_ippu_production = sf.vec_bounds(array_ippu_production, (0, np. inf))

        # get production in terms of output variable (should be 1, and add net imports and production to output dataframe)
        array_ippu_production *= self.model_attributes.get_mass_equivalent(
            self.model_attributes.get_variable_characteristic(self.modvar_ippu_prod_qty_init, "$UNIT-MASS$"),
            self.model_attributes.get_variable_characteristic(self.modvar_ippu_qty_total_production, "$UNIT-MASS$")
        )
        df_out += [
            self.model_attributes.array_to_df(array_ippu_change_net_imports, self.modvar_ippu_change_net_imports, False, True),
            self.model_attributes.array_to_df(array_ippu_production, self.modvar_ippu_qty_total_production, False, True)
        ]


        ##  USE EMISSION FACTORS TO ESTIMATE EMISSIONS FROM PROCESSES AND PRODUCT USE - LOOP OVER A DICTIONARY GIVEN THE NUMBER OF FACTORS

        # dictionary variables mapping emission variable to factor tuples (gdp, production). No factor gives ""
        dict_ippu_simple_efs = {
            self.modvar_ippu_emissions_ch4: ("", self.modvar_ippu_ef_ch4_per_prod),
            self.modvar_ippu_emissions_co2: (self.modvar_ippu_ef_co2_per_gdp, self.modvar_ippu_ef_co2_per_prod),
            self.modvar_ippu_emissions_hfc: (self.modvar_ippu_ef_hfc_per_gdp, self.modvar_ippu_ef_hfc_per_prod),
            self.modvar_ippu_emissions_n2o: (self.modvar_ippu_ef_n2o_per_gdp, self.modvar_ippu_ef_n2o_per_prod),
            self.modvar_ippu_emissions_nf3: ("", self.modvar_ippu_ef_nf3_per_prod),
            self.modvar_ippu_emissions_pfc: (self.modvar_ippu_ef_pfc_per_gdp, self.modvar_ippu_ef_pfc_per_prod),
            self.modvar_ippu_emissions_sf6: (self.modvar_ippu_ef_sf6_per_gdp, self.modvar_ippu_ef_sf6_per_prod)
        }

        # process is identical across emission factors -- sum gdp-driven and production-driven factors
        for modvar in dict_ippu_simple_efs.keys():
            # get variables and initialize total emissions
            modvar_ef_gdp = dict_ippu_simple_efs[modvar][0]
            modvar_ef_prod = dict_ippu_simple_efs[modvar][1]
            array_ippu_emission = np.zeros((len(df_ippu_trajectories), attr_ippu.n_key_values))
            # check if there is a gdp driven factor
            if modvar_ef_gdp != "":
                array_ippu_emission_cur = self.model_attributes.get_standard_variables(df_ippu_trajectories, modvar_ef_gdp, False, return_type = "array_units_corrected", expand_to_all_cats = True)
                array_ippu_emission += (array_ippu_emission_cur.transpose()*vec_gdp).transpose()
            # check if there is a production driven factor
            if modvar_ef_prod != "":
                scalar_ippu_mass = self.model_attributes.get_mass_equivalent(
                    self.model_attributes.get_variable_characteristic(self.modvar_ippu_qty_total_production, "$UNIT-MASS$"),
                    self.model_attributes.get_variable_characteristic(modvar_ef_prod, "$UNIT-MASS$")
                )
                array_ippu_emission_cur = self.model_attributes.get_standard_variables(df_ippu_trajectories, modvar_ef_prod, False, return_type = "array_units_corrected", expand_to_all_cats = True)
                array_ippu_emission += array_ippu_emission_cur*array_ippu_production
            # add to output dataframe
            df_out += [
                self.model_attributes.array_to_df(array_ippu_emission, modvar, False, True)
            ]


        # concatenate and add subsector emission totals
        df_out = sf.merge_output_df_list(df_out, self.model_attributes, "concatenate")
        self.model_attributes.add_subsector_emissions_aggregates(df_out, self.required_base_subsectors, False)

        return df_out







In [257]:

#########################
###                   ###
###     IPPU MODEL    ###
###                   ###
#########################

class IPPU:

    def __init__(self, attributes: ds.ModelAttributes):

        self.model_attributes = attributes
        self.required_dimensions = self.get_required_dimensions()
        self.required_subsectors, self.required_base_subsectors = self.get_required_subsectors()
        self.required_variables, self.output_variables = self.get_ippu_input_output_fields()

        ##  set some model fields to connect to the attribute tables

        # ippu model variables
        self.modvar_ippu_change_net_imports = "Change to Net Imports of Recyclable Products"
        self.modvar_ippu_ef_ch4_per_prod = ":math:\\text{CH}_4 Production Emission Factor"
        self.modvar_ippu_ef_co2_per_gdp = ":math:\\text{CO}_2 GDP Emission Factor"
        self.modvar_ippu_ef_co2_per_prod = ":math:\\text{CO}_2 Production Emission Factor"
        self.modvar_ippu_ef_hfc_per_gdp = "HFC Emission GDP Factor Placeholder"
        self.modvar_ippu_ef_hfc_per_prod = "HFC Emission Production Factor Placeholder"
        self.modvar_ippu_ef_n2o_per_gdp = ":math:\\text{N}_2\\text{O} GDP Emission Factor"
        self.modvar_ippu_ef_n2o_per_prod = ":math:\\text{N}_2\\text{O} Production Emission Factor"
        self.modvar_ippu_ef_nf3_per_prod = ":math:\\text{NF}_3 Production Emission Factor"
        self.modvar_ippu_ef_pfc_per_gdp = "PFC Emission GDP Factor Placeholder"
        self.modvar_ippu_ef_pfc_per_prod = "PFC Emission Production Factor Placeholder"
        self.modvar_ippu_ef_sf6_per_gdp = ":math:\\text{SF}_6 GDP Emission Factor"
        self.modvar_ippu_ef_sf6_per_prod = ":math:\\text{SF}_6 Production Emission Factor"
        self.modvar_ippu_elast_ind_prod_to_gdp = "Elasticity of Industrial Production to GDP"
        self.modvar_ippu_emissions_ch4 = ":math:\\text{CH}_4 Emissions from Industry"
        self.modvar_ippu_emissions_co2 = ":math:\\text{CO}_2 Emissions from Industry"
        self.modvar_ippu_emissions_hfc = "HFC Emissions Placeholder"
        self.modvar_ippu_emissions_n2o = ":math:\\text{N}_2\\text{O} Emissions from Industry"
        self.modvar_ippu_emissions_pfc = "PFC Emissions Placeholder"
        self.modvar_ippu_emissions_nf3 = ":math:\\text{NF}_3 Emissions from Industry"
        self.modvar_ippu_emissions_sf6 = ":math:\\text{SF}_6 Emissions from Industry"
        self.modvar_ippu_prod_qty_init = "Initial Industrial Production"
        self.modvar_ippu_wwf_cod = "COD Wastewater Factor"
        self.modvar_ippu_wwf_vol = "Wastewater Production Factor"
        self.modvar_ippu_qty_total_production = "Industrial Production"


        ##  MISCELLANEOUS VARIABLES
        self.time_periods, self.n_time_periods = self.model_attributes.get_time_periods()

        # add other model classes
        self.model_circecon = CircularEconomy(self.model_attributes)
        self.model_socioeconomic = Socioeconomic(self.model_attributes)


    ##  FUNCTIONS FOR MODEL ATTRIBUTE DIMENSIONS

    def check_df_fields(self, df_ippu_trajectories):
        check_fields = self.required_variables
        # check for required variables
        if not set(check_fields).issubset(df_ippu_trajectories.columns):
            set_missing = list(set(check_fields) - set(df_ippu_trajectories.columns))
            set_missing = sf.format_print_list(set_missing)
            raise KeyError(f"IPPU projection cannot proceed: The fields {set_missing} are missing.")


    def get_required_subsectors(self):
        subsectors = list(sf.subset_df(self.model_attributes.dict_attributes["abbreviation_subsector"].table, {"sector": ["IPPU"]})["subsector"])
        subsectors_base = subsectors.copy()
        subsectors += ["Economy", "General"]
        return subsectors, subsectors_base

    def get_required_dimensions(self):
        ## TEMPORARY - derive from attributes later
        required_doa = [self.model_attributes.dim_time_period]
        return required_doa

    def get_ippu_input_output_fields(self):
        required_doa = [self.model_attributes.dim_time_period]
        required_vars, output_vars = self.model_attributes.get_input_output_fields(self.required_subsectors)
        return required_vars + self.get_required_dimensions(), output_vars



    ######################################
    #    SUBSECTOR SPECIFIC FUNCTIONS    #
    ######################################

    
    # project industrial production—broken out so that other sectors can call it
    def project_industrial_production(self, 
        df_ippu_trajectories: pd.DataFrame,
        vec_rates_gdp: np.ndarray,
        dict_dims: dict = None,
        n_projection_time_periods: int = None, 
        projection_time_periods: list = None            
    ) -> np.ndarray:
        """
            project_industrial_production can be called from other sectors to simplify calculation of industrial production. If any of dict_dims, n_projection_time_periods, or projection_time_periods are unspecified (expected if ran outside of IPPU.project()), self.model_attributes.check_projection_input_df wil be run
            
            dict_dims: dict of dimensions (returned from check_projection_input_df). Default is None. 
            
            n_projection_time_periods: integer giving number of time periods (returned from check_projection_input_df). Default is None. 
            
            projection_time_periods: list of time periods (returned from check_projection_input_df). Default is None. 
        """
        # allows production to be run outside of the project method
        if type(None) in set([type(x) for x in [dict_dims, n_projection_time_periods, projection_time_periods]]):
            dict_dims, df_ippu_trajectories, n_projection_time_periods, projection_time_periods = self.model_attributes.check_projection_input_df(df_ippu_trajectories, True, True, True)
        
        # get initial production and apply elasticities to gdp to calculate growth in production
        array_ippu_prod_init_by_cat = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.modvar_ippu_prod_qty_init, False, return_type = "array_base", var_bounds = (0, np.inf), expand_to_all_cats = True)
        array_ippu_elasticity_prod_to_gdp = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.modvar_ippu_elast_ind_prod_to_gdp, False, return_type = "array_base", expand_to_all_cats = True)        
        array_ippu_ind_growth = sf.project_growth_scalar_from_elasticity(vec_rates_gdp, array_ippu_elasticity_prod_to_gdp, False, "standard")
        array_ippu_ind_prod = array_ippu_prod_init_by_cat[0]*array_ippu_ind_growth
        
        return array_ippu_ind_prod
        
        
    # project method for IPPU
    def project(self, df_ippu_trajectories: pd.DataFrame) -> pd.DataFrame:
        
        # make sure socioeconomic variables are added and
        df_ippu_trajectories, df_se_internal_shared_variables = self.model_socioeconomic.project(df_ippu_trajectories)
        # check that all required fields are contained—assume that it is ordered by time period
        self.check_df_fields(df_ippu_trajectories)
        dict_dims, df_ippu_trajectories, n_projection_time_periods, projection_time_periods = self.model_attributes.check_projection_input_df(df_ippu_trajectories, True, True, True)
        
        ##  CATEGORY AND ATTRIBUTE INITIALIZATION
        pycat_gnrl = self.model_attributes.get_subsector_attribute("General", "pycategory_primary")
        pycat_ippu = self.model_attributes.get_subsector_attribute("IPPU", "pycategory_primary")
        pycat_waso = self.model_attributes.get_subsector_attribute("Solid Waste", "pycategory_primary")
        # attribute tables
        attr_gnrl = self.model_attributes.dict_attributes[pycat_gnrl]
        attr_ippu = self.model_attributes.dict_attributes[pycat_ippu]
        attr_waso = self.model_attributes.dict_attributes[pycat_waso]


        ##  ECON/GNRL VECTOR AND ARRAY INITIALIZATION

        # get some vectors
        vec_gdp = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_econ_gdp, False, return_type = "array_base")
        vec_pop = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_gnrl_pop_total, False, return_type = "array_base")
        array_pop = self.model_attributes.get_standard_variables(df_ippu_trajectories, self.model_socioeconomic.modvar_gnrl_subpop, False, return_type = "array_base")
        vec_gdp_per_capita = np.array(df_se_internal_shared_variables["vec_gdp_per_capita"])
        vec_rates_gdp = np.array(df_se_internal_shared_variables["vec_rates_gdp"].dropna())
        vec_rates_gdp_per_capita = np.array(df_se_internal_shared_variables["vec_rates_gdp_per_capita"].dropna())
        
        
        ##  OUTPUT INITIALIZATION
        df_out = [df_ippu_trajectories[self.required_dimensions].copy()]
        
        
        ######################################################
        #    INDUSTRIAL PRODUCTION + RECYCLING ADJUSTMENT    #
        ######################################################
        
        # initialize production + initialize change to net imports as 0 (reduce categories later)
        array_ippu_production = self.project_industrial_production(df_ippu_trajectories, vec_rates_gdp, dict_dims, n_projection_time_periods, projection_time_periods)
        array_ippu_change_net_imports = np.zeros((array_ippu_production.shape[0], attr_ippu.n_key_values))
        
                                                 
        ##  PERFORM THE RECYCLING ADJUSTMENT (if recycling data are provided from the waste model)
                                                 
        array_ippu_recycled = self.model_attributes.get_optional_or_integrated_standard_variable(df_ippu_trajectories, self.model_circecon.modvar_waso_waste_total_recycled, None, True, "array_base")
        if type(array_ippu_recycled) != type(None):
            # if recycling totals are passed from the waste model, convert to ippu categories
            cats_waso_recycle = sa.model_attributes.get_variable_categories(self.model_circecon.modvar_waso_waste_total_recycled)
            dict_repl = attr_waso.field_maps[f"{pycat_waso}_to_{pycat_ippu}"]
            cats_ippu_recycle = [ds.clean_schema(dict_repl[x]) for x in cats_waso_recycle]
            array_ippu_recycled_waste = self.model_attributes.merge_array_var_partial_cat_to_array_all_cats(
                array_ippu_recycled[1], 
                None,
                output_cats = cats_ippu_recycle,
                output_subsec = "IPPU"
            )
            # units correction to ensure consistency from waso -> ippu
            factor_ippu_waso_recycle_to_ippu_recycle = self.model_attributes.get_mass_equivalent(
                self.model_attributes.get_variable_characteristic(self.model_circecon.modvar_waso_waste_total_recycled, "$UNIT-MASS$"),
                self.model_attributes.get_variable_characteristic(self.modvar_ippu_prod_qty_init, "$UNIT-MASS$")
            )
            array_ippu_recycled_waste *= factor_ippu_waso_recycle_to_ippu_recycle
            array_ippu_production += array_ippu_recycled_waste
            # next, check for industrial categories whose production is affected by recycling, then adjust downwards
            vec_ippu_cats_to_adjust_from_recycling = [ds.clean_schema(x) for x in sa.model_attributes.get_ordered_category_attribute("IPPU", "target_recycling_cat_industry_to_adjust")]
            w = [i for i in range(len(vec_ippu_cats_to_adjust_from_recycling)) if (vec_ippu_cats_to_adjust_from_recycling[i] != "none") and (vec_ippu_cats_to_adjust_from_recycling[i] in attr_ippu.key_values)]
            if len(w) > 0:
                array_ippu_recycled_waste_adj = array_ippu_recycled_waste[:, w].copy()
                array_ippu_recycled_waste_adj = self.model_attributes.merge_array_var_partial_cat_to_array_all_cats(
                    array_ippu_recycled_waste_adj, 
                    None,
                    output_cats = np.array(vec_ippu_cats_to_adjust_from_recycling)[w],
                    output_subsec = "IPPU"
                )
                # inititialize production, then get change to net imports (anything negative) and reduce virgin production accordingly
                array_ippu_production = array_ippu_production - array_ippu_recycled_waste_adj
                array_ippu_change_net_imports = sf.vec_bounds(array_ippu_production, (-np.inf, 0))
                array_ippu_production = sf.vec_bounds(array_ippu_production, (0, np. inf))
        
        # get production in terms of output variable (should be 1, and add net imports and production to output dataframe)
        array_ippu_production *= self.model_attributes.get_mass_equivalent(
            self.model_attributes.get_variable_characteristic(self.modvar_ippu_prod_qty_init, "$UNIT-MASS$"),
            self.model_attributes.get_variable_characteristic(self.modvar_ippu_qty_total_production, "$UNIT-MASS$")
        )
        df_out += [
            self.model_attributes.array_to_df(array_ippu_change_net_imports, self.modvar_ippu_change_net_imports, False, True),
            self.model_attributes.array_to_df(array_ippu_production, self.modvar_ippu_qty_total_production, False, True)
        ]
                                                 
        
        ##  USE EMISSION FACTORS TO ESTIMATE EMISSIONS FROM PROCESSES AND PRODUCT USE - LOOP OVER A DICTIONARY GIVEN THE NUMBER OF FACTORS
        
        # dictionary variables mapping emission variable to factor tuples (gdp, production). No factor gives ""
        dict_ippu_simple_efs = {
            self.modvar_ippu_emissions_ch4: ("", self.modvar_ippu_ef_ch4_per_prod),
            self.modvar_ippu_emissions_co2: (self.modvar_ippu_ef_co2_per_gdp, self.modvar_ippu_ef_co2_per_prod),
            self.modvar_ippu_emissions_hfc: (self.modvar_ippu_ef_hfc_per_gdp, self.modvar_ippu_ef_hfc_per_prod),
            self.modvar_ippu_emissions_n2o: (self.modvar_ippu_ef_n2o_per_gdp, self.modvar_ippu_ef_n2o_per_prod),
            self.modvar_ippu_emissions_nf3: ("", self.modvar_ippu_ef_nf3_per_prod),
            self.modvar_ippu_emissions_pfc: (self.modvar_ippu_ef_pfc_per_gdp, self.modvar_ippu_ef_pfc_per_prod),
            self.modvar_ippu_emissions_sf6: (self.modvar_ippu_ef_sf6_per_gdp, self.modvar_ippu_ef_sf6_per_prod)
        }
        
        # process is identical across emission factors -- sum gdp-driven and production-driven factors
        for modvar in dict_ippu_simple_efs.keys():
            # get variables and initialize total emissions
            modvar_ef_gdp = dict_ippu_simple_efs[modvar][0]
            modvar_ef_prod = dict_ippu_simple_efs[modvar][1]
            array_ippu_emission = np.zeros((len(df_ippu_trajectories), attr_ippu.n_key_values))
            # check if there is a gdp driven factor
            if modvar_ef_gdp != "":
                array_ippu_emission_cur = self.model_attributes.get_standard_variables(df_ippu_trajectories, modvar_ef_gdp, False, return_type = "array_units_corrected", expand_to_all_cats = True)
                array_ippu_emission += (array_ippu_emission_cur.transpose()*vec_gdp).transpose()
            # check if there is a production driven factor
            if modvar_ef_prod != "":
                scalar_ippu_mass = self.model_attributes.get_mass_equivalent(
                    self.model_attributes.get_variable_characteristic(self.modvar_ippu_qty_total_production, "$UNIT-MASS$"),
                    self.model_attributes.get_variable_characteristic(modvar_ef_prod, "$UNIT-MASS$")
                )
                array_ippu_emission_cur = self.model_attributes.get_standard_variables(df_ippu_trajectories, modvar_ef_prod, False, return_type = "array_units_corrected", expand_to_all_cats = True)
                array_ippu_emission += array_ippu_emission_cur*array_ippu_production
            # add to output dataframe
            df_out += [
                self.model_attributes.array_to_df(array_ippu_emission, modvar, False, True)
            ]

        
        # concatenate and add subsector emission totals
        df_out = sf.merge_output_df_list(df_out, self.model_attributes, "concatenate")
        self.model_attributes.add_subsector_emissions_aggregates(df_out, self.required_base_subsectors, False)   
        
        return df_out
    

    
    
model_ippu = IPPU(sa.model_attributes)
df_project_ippu = model_ippu.project(df_waste_integrated)




/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1284: UserWarning: clean_partial_category_dictionary: Invalid categories values 'wood' dropped when cleaning the dictionary. Category values not found.
  warnings.warn(f"clean_partial_category_dictionary: Invalid categories values {missing_vals} dropped when cleaning the dictionary. Category values not found.")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1284: UserWarning: clean_partial_category_dictionary: Invalid categories values 'wood' dropped when cleaning the dictionary. Category values not found.
  warnings.warn(f"clean_partial_category_dictionary: Invalid categories values {missing_vals} dropped when cleaning the dictionary. Category values not found.")


In [259]:
df_project_ippu[[x for x in df_project_ippu.columns if "emission_co2e" in x]]



,emission_co2e_ch4_ippu_chemicals,emission_co2e_ch4_ippu_metals,emission_co2e_co2_ippu_cement,emission_co2e_co2_ippu_chemicals,emission_co2e_co2_ippu_glass,emission_co2e_co2_ippu_lime_and_carbonite,emission_co2e_co2_ippu_metals,emission_co2e_co2_ippu_non_energy_products,emission_co2e_co2_ippu_recycled_glass,emission_co2e_hfc134a_ippu_chemicals,...,emission_co2e_hfc23_ippu_ods_products,emission_co2e_hfc23_ippu_other_product_manufacturing,emission_co2e_n2o_ippu_chemicals,emission_co2e_n2o_ippu_other_product_manufacturing,emission_co2e_nf3_ippu_chemicals,emission_co2e_nf3_ippu_electronics,emission_co2e_sf6_ippu_chemicals,emission_co2e_sf6_ippu_electronics,emission_co2e_sf6_ippu_other_product_manufacturing,emission_co2e_subsector_total_ippu
0,2.232000,0.204529,0.100000,0.080000,0.010422,0.001400,0.007331,0.000055,0.003578,0.002448,...,0.000016,0.000080,21.840000,0.014969,2016.000000,126.000000,2016.000000,126.000000,1.381714,4310.248762
1,2.242418,0.203928,0.100242,0.080373,0.010391,0.001401,0.007309,0.000057,0.003633,0.002459,...,0.000017,0.000083,21.941939,0.015604,2025.409721,126.250172,2025.409721,126.250172,1.440381,4329.741127
2,2.250792,0.203317,0.100436,0.080674,0.010360,0.001403,0.007287,0.000059,0.003683,0.002469,...,0.000017,0.000086,22.023880,0.016134,2032.973501,126.450730,2032.973501,126.450730,1.489314,4345.410166
3,2.257309,0.202702,0.100586,0.080907,0.010329,0.001403,0.007265,0.000061,0.003729,0.002476,...,0.000018,0.000089,22.087649,0.016559,2038.859935,126.606479,2038.859935,126.606479,1.528542,4357.604758
4,2.264565,0.202125,0.100754,0.081167,0.010300,0.001404,0.007245,0.000062,0.003775,0.002484,...,0.000018,0.000091,22.158648,0.017043,2045.413701,126.779597,2045.413701,126.779597,1.573239,4371.182409
5,2.271576,0.201559,0.100915,0.081418,0.010272,0.001405,0.007224,0.000064,0.003820,0.002491,...,0.000019,0.000094,22.227248,0.017523,2051.745939,126.946555,2051.745939,126.946555,1.617545,4384.301619
6,2.279105,0.201010,0.101089,0.081688,0.010244,0.001406,0.007205,0.000066,0.003865,0.002500,...,0.000019,0.000097,22.300922,0.018052,2058.546642,127.125547,2058.546642,127.125547,1.666318,4398.392036
7,2.286913,0.200474,0.101268,0.081968,0.010216,0.001407,0.007185,0.000068,0.003910,0.002508,...,0.000020,0.000100,22.377318,0.018614,2065.598565,127.310798,2065.598565,127.310798,1.718245,4413.003659
8,2.295300,0.199953,0.101461,0.082269,0.010190,0.001409,0.007167,0.000070,0.003956,0.002517,...,0.000021,0.000103,22.459388,0.019235,2073.174257,127.509419,2073.174257,127.509419,1.775572,4428.701378
9,2.304223,0.199445,0.101665,0.082589,0.010164,0.001410,0.007149,0.000073,0.004003,0.002527,...,0.000021,0.000107,22.546698,0.019916,2081.233643,127.720277,2081.233643,127.720277,1.838362,4445.402360


In [152]:
attr_cur = sa.model_attributes.get_attribute_table("Solid Waste")

[x for x in attr_cur.key_values if x in list(attr_cur.table[attr_cur.key])]


['chemical_industrial',
 'food',
 'glass',
 'metal',
 'nappies',
 'other',
 'paper',
 'plastic',
 'rubber_leather',
 'sludge',
 'textiles',
 'wood',
 'yard']

In [109]:
array_ippu_recycled = sa.model_attributes.get_optional_or_integrated_standard_variable(df_project_ce, model_circecon.modvar_waso_waste_total_recycled, None, True, "array_base")
array_ippu_recycled
    


('Total Waste Recycled',
 array([[ 3577.89750318,  2669.22512142, 14198.005965  , 13630.0857264 ,
           340.75214316,  2214.88893054,   170.37607158],
        [ 3633.01871863,  2710.34729803, 14457.41151389, 13840.07130908,
           346.00178273,  2240.55210981,   172.51284456],
        [ 3683.43069887,  2747.95623566, 14688.78568232, 14032.11694806,
           350.8029237 ,  2265.28558746,   174.53957243],
        [ 3729.28507983,  2782.16505956, 14893.24827081, 14206.80030411,
           355.1700076 ,  2289.04330951,   176.45566954],
        [ 3775.19928132,  2816.41851146, 15103.30755919, 14381.71154787,
           359.5427887 ,  2311.75463274,   178.31185337],
        [ 3819.89917789,  2849.76605335, 15308.23848379, 14551.99686816,
           363.7999217 ,  2333.79983384,   180.11497886],
        [ 3865.0893993 ,  2883.47939313, 15519.03625568, 14724.15009258,
           368.10375231,  2355.37240777,   181.89635362],
        [ 3910.26111253,  2917.17892522, 15732.4115617 , 1

In [48]:
array1 = sa.model_attributes.get_standard_variables(df_complete, ":math:\\text{CH}_4 Production Emission Factor", False, return_type = "array_base")

array2 = sa.model_attributes.merge_array_var_partial_cat_to_array_all_cats(array1, ":math:\\text{CH}_4 Production Emission Factor")
        
    
array2


array([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.,